### Note: <br>
This code for the fine-tuning process of XLNet for hate speech detection is based on the following example, published on Medium: <br>

link to article: https://medium.com/swlh/using-xlnet-for-sentiment-classification-cfa948e65e85 <br>
author: Shanay Ghag <br>
published at: Jun 16, 2020<br>
link to GitHub: https://github.com/shanayghag/Sentiment-classification-using-XLNet

In [10]:
from datasets import load_dataset_builder, load_dataset
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import re

import sentencepiece
from collections import defaultdict

import torch
from torch import nn, optim
from torch.utils.data import TensorDataset,RandomSampler,SequentialSampler, Dataset, DataLoader,random_split,SubsetRandomSampler
import torch.nn.functional as F
from keras.preprocessing.sequence import pad_sequences
import transformers
from transformers import XLNetTokenizer, XLNetModel, AdamW, get_linear_schedule_with_warmup
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from sklearn.model_selection import KFold

In [11]:
# set up connection to drive: 
#from google.colab import drive
#drive.mount('/content/drive')

# define device: 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [12]:
def prepare_text(text):
    text = re.sub(r"@[A-Za-z0-9_]+", ' ', text) # remove @user 
    text = re.sub(r"https?://[A-Za-z0-9./]+", ' ', text) # remove links
    text = re.sub(r"[^a-zA-z.!?'0-9]", ' ', text) # remove smileys
    text = re.sub('#', '', text) # remove hash sign
    text = re.sub('\t', ' ',  text) # remove tab
    text = re.sub(r" +", ' ', text) # remove multiple whitespaces
    return text

# 1. Prepare Hate Dataset for Fine-Tuning

In [2]:
# load dataset from hugging face hub: tweets_hate_speech_detection
hate_data_train = load_dataset('tweets_hate_speech_detection', split='train')

Generating train split:   0%|          | 0/31962 [00:00<?, ? examples/s]

Dataset tweets_hate_speech_detection downloaded and prepared to /root/.cache/huggingface/datasets/tweets_hate_speech_detection/default/0.0.0/c6b6f41e91ac9113e1c032c5ecf7a49b4e1e9dc8699ded3c2d8425c9217568b2. Subsequent calls will reuse this data.


In [13]:
tweet = hate_data_train['tweet']
label = hate_data_train['label']
df_hate_1 = pd.DataFrame({'text': tweet, 'label': label}) # 0 = no hate; 1 = hate (racist or sexist)


In [3]:
# load different hate speech dataset:
hate_data_2= load_dataset('hate_speech_offensive', split='train')

Generating train split:   0%|          | 0/24783 [00:00<?, ? examples/s]

Dataset hate_speech_offensive downloaded and prepared to /root/.cache/huggingface/datasets/hate_speech_offensive/default/1.0.0/5f5dfc7b42b5c650fe30a8c49df90b7dbb9c7a4b3fe43ae2e66fabfea35113f5. Subsequent calls will reuse this data.


In [5]:
text_2 = hate_data_2['tweet']
label_2 = hate_data_2['class']

df_hate_2 = pd.DataFrame({'text': text_2, 'label': label_2}) # 0 = hate-speech; 1 = offensive-language; 2 = neither

In [6]:
# change label to match df_hate_1 and binary classification: 0: no hate; 1: hate
df_hate_2.loc[(df_hate_2.label == 0),'label']=1 # change 0 to 1 to collapse hate speech and offensive language in one class
df_hate_2.loc[(df_hate_2.label == 2),'label']=0 # then change 2 to 0 to match df_hate_1 classes
# 0 = no hate; 1: hate 

In [9]:
df_hate = pd.concat([df_hate_1, df_hate_2])

In [38]:
df_hate = shuffle(df_hate)
df_hate = df_hate[:24000]
df_hate.head()

,text,label
7439,used the write stuff best piece of writing al...,0
25259,in pueorico environmental injustice and inflam...,1
19175,RT Kim K got bitches thinkin being a hoe is a ...,1
25552,no words . . . self selfie porait selfporait f...,0
11315,I'm late as hell....Niggas really hating cuz t...,1


In [14]:
def dataset_preprocessing(df_hate):
    df_hate['text'] = df_hate['text'].apply(prepare_text)
    print(f'non-hate tweets: {len(df_hate[df_hate["label"] == 0])}')
    print(f'hate tweets: {len(df_hate[df_hate["label"] == 1])}')
    return df_hate

# 2. Load XLNet, Prepare Inputs and Define Hyperparameter

In [17]:
# custom dataset class
class HateDataset(Dataset):
    def __init__(self, reviews, targets, tokenizer, max_len):
        self.reviews = reviews
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)
    
    def __getitem__(self, item):
        review = str(self.reviews[item])
        target = self.targets[item]
        
        encoding = self.tokenizer.encode_plus(review,
                                              add_special_tokens=True,
                                              max_length=self.max_len,
                                              truncation=True,
                                              return_token_type_ids=False,
                                              pad_to_max_length=False,
                                              return_attention_mask=True,
                                              return_tensors='pt',)
        
        input_ids = pad_sequences(encoding['input_ids'], 
                                  maxlen=MAX_LEN, 
                                  dtype=torch.Tensor ,
                                  truncating="post",
                                  padding="post")
        input_ids = input_ids.astype(dtype = 'int64')
        input_ids = torch.tensor(input_ids) 
        
        attention_mask = pad_sequences(encoding['attention_mask'], 
                                       maxlen=MAX_LEN, 
                                       dtype=torch.Tensor ,
                                       truncating="post",
                                       padding="post")
        attention_mask = attention_mask.astype(dtype = 'int64')
        attention_mask = torch.tensor(attention_mask)       
        
        return {'review_text': review,
                'input_ids': input_ids,
                'attention_mask': attention_mask.flatten(),
                'targets': torch.tensor(target, dtype=torch.long)}

In [18]:
# define custom dataloader:
def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = HateDataset(reviews=df.text.to_numpy(),
                     targets=df.label.to_numpy(),
                     tokenizer=tokenizer,
                     max_len=max_len)
    
    return DataLoader(ds,
                    batch_size=batch_size,
                    num_workers=2)                    

In [19]:
# define training step function:
from sklearn import metrics

def train_epoch(model, data_loader, optimizer, device, scheduler, n_examples, progress_bar):
    model = model.train()
    losses = []
    acc = 0
    counter = 0
    
    for d in data_loader:
        input_ids = d["input_ids"].reshape(8,512).to(device)
        attention_mask = d["attention_mask"].to(device)
        targets = d["targets"].to(device)
    
        outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)
        loss = outputs[0]
        logits = outputs[1]
    
        _, prediction = torch.max(outputs[1], dim=1)
        targets = targets.cpu().detach().numpy()
        prediction = prediction.cpu().detach().numpy()
        accuracy = metrics.accuracy_score(targets, prediction)

        acc += accuracy
        losses.append(loss.item())

        loss.backward()

        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        counter = counter + 1
    
    return acc / counter, np.mean(losses)

In [20]:
# define evaluation function:
def eval_model(model, data_loader, device, n_examples):
    model = model.eval()
    losses = []
    acc = 0
    counter = 0
    
    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].reshape(8,512).to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)
            
            outputs = model(input_ids=input_ids, token_type_ids=None, attention_mask=attention_mask, labels = targets)
            loss = outputs[0]
            logits = outputs[1]
            
            _, prediction = torch.max(outputs[1], dim=1)
            targets = targets.cpu().detach().numpy()
            prediction = prediction.cpu().detach().numpy()
            accuracy = metrics.accuracy_score(targets, prediction)
            
            acc += accuracy
            losses.append(loss.item())
            counter += 1
            
    return acc / counter, np.mean(losses)

In [ ]:
# read model saved in previous step:
# when connected to GPU:
# model.load_state_dict(torch.load('xlnet_model_hate.bin'))

# when not connected to GPU (hence no cuda):
#model.load_state_dict(torch.load('/content/drive/MyDrive/Colab Notebooks/models/xlnet_model_hate.bin', map_location=torch.device('cpu')))

In [21]:
# define function to get prediction of data
def get_predictions(model, data_loader):
    model = model.eval()
    
    review_texts = []
    predictions = []
    prediction_probs = []
    real_values = []
    
    with torch.no_grad():
        for d in data_loader:
            texts = d["review_text"]
            input_ids = d["input_ids"].reshape(8,512).to(device)
            attention_mask = d["attention_mask"].to(device)
            targets = d["targets"].to(device)
            
            outputs = model(input_ids=input_ids,
                            token_type_ids=None,
                            attention_mask=attention_mask,#
                            labels=targets)
            
            loss = outputs[0]
            logits = outputs[1]
            
            _, preds = torch.max(outputs[1], dim=1)
            probs = F.softmax(outputs[1], dim=1)
            
            review_texts.extend(texts)
            predictions.extend(preds)
            prediction_probs.extend(probs)
            real_values.extend(targets)
            
    predictions = torch.stack(predictions).cpu()
    prediction_probs = torch.stack(prediction_probs).cpu()
    real_values = torch.stack(real_values).cpu()
    
    return review_texts, predictions, prediction_probs, real_values

In [22]:
def show_confusion_matrix(confusion_matrix):
    hmap = sns.heatmap(confusion_matrix/ confusion_matrix.sum(axis=1)[:, np.newaxis], annot=True, fmt='.2%', cmap="Blues")
    hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
    hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')

    plt.ylabel('True Hate')
    plt.xlabel('Predicted Hate')
    plt.savefig('confusion_hate');
    



In [24]:
# load pre-trained XLNet model
from transformers import XLNetForSequenceClassification

xlnet_model = XLNetForSequenceClassification.from_pretrained('xlnet-base-cased', num_labels = 2)

tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'logits_proj.weight', 'sequence_summary.summary.bias', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

non-hate tweets: 14288
hate tweets: 9712
                                                    text  label
11174   I'll go to class the last 30 minutes of that hoe      1
7294                        yeah wtf you aren't a pussy.      1
9263                   Fuck niggah add it up.. gt gt gt       1
19411         independenceday lee hak buffet restaurant       0
21997  it's never too late never too late to sta over...      0
                                                    text  label
20301  furniture purchased home demo on friday so car...      0
11174   I'll go to class the last 30 minutes of that hoe      1
7294                        yeah wtf you aren't a pussy.      1
5827    she can just fuckin call the venue about it a...      1
9263                   Fuck niggah add it up.. gt gt gt       1
                                                    text  label
20301  furniture purchased home demo on friday so car...      0
11174   I'll go to class the last 30 minutes of that hoe      1

text     some bitches care more about their eyebrows th...
label                                                    0
Name: 271, dtype: object

In [ ]:
report_lst = []
df_hate = dataset_preprocessing(df_hate)
df_train, df_test = train_test_split(df_hate, test_size=0.25, random_state=101)
splits = KFold(n_splits=5, shuffle=True, random_state=42)  # k-fold k=5 as in the BABE paper


for fold, (train_ids, val_ids) in enumerate(splits.split(np.arange(len(df_train)))):
    df_train.reset_index(drop = True)
    df_training = df_train.iloc[train_ids]
    df_val = df_train.iloc[val_ids]
    
    model = copy.deepcopy(xlnet_model)
    model.to(device)
    MAX_LEN = 512
    # create training, validation and test set: 
    
    #df_val, df_test = train_test_split(df_test, test_size=0.5, random_state=101)
    # define batch size: 
    BATCH_SIZE = 8

    # create data loader for training, validation and test set: 
    train_data_loader = create_data_loader(df_training, tokenizer, MAX_LEN, BATCH_SIZE)
    val_data_loader = create_data_loader(df_val, tokenizer, MAX_LEN, BATCH_SIZE)
    test_data_loader = create_data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)
    
    # defining hyperparameters:
    EPOCHS = 2

    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [{'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
                                    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay':0.0}]

    optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5)
    total_steps = len(train_data_loader) * EPOCHS

    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0,
                                                num_training_steps=total_steps)
    history = defaultdict(list)
    best_accuracy = 0
    num_training_steps = EPOCHS * len(train_data_loader)
    progress_bar = tqdm(range(num_training_steps))

    for epoch in range(EPOCHS):
        print(f'Epoch {epoch + 1}/{EPOCHS}')
        print('-' * 10)

        # call train function
        train_acc, train_loss = train_epoch(model,
                                          train_data_loader,
                                          optimizer,
                                          device, 
                                          scheduler, 
                                          len(df_train),
                                            progress_bar)

        print(f'Train loss {train_loss} Train accuracy {train_acc}')

        # call evaluation function
        val_acc, val_loss = eval_model(model,
                                     val_data_loader, 
                                     device, 
                                     len(df_val))

        print(f'Val loss {val_loss} Val accuracy {val_acc}')
        print()

        history['train_acc'].append(train_acc)
        history['train_loss'].append(train_loss)
        history['val_acc'].append(val_acc)
        history['val_loss'].append(val_loss)

        if val_acc > best_accuracy:
            # torch.save(model.state_dict(), 'xlnet_model_hate.bin') # note: this scrip was implemented using Kaggle's GPU. The model was saved into the Kaggle repo and downloaded manually
            best_accuracy = val_acc
    fig = plt.figure()
    plt.plot(history['train_acc'], label='train accuracy')
    plt.plot(history['val_acc'], label='validation accuracy')
    plt.plot(history['train_loss'], label='train loss')
    plt.plot(history['val_loss'], label='validation loss')
    #plt.title('Training History')
    plt.ylabel('Accuracy and Loss')
    plt.xlabel('Epoch')
    plt.legend()
    plt.ylim([0, 1])
    # fig.savefig('training_hate.png');
    plt.show()
    
    # EVALUATION
    # call evaluation function and apply to test set
    test_acc, test_loss = eval_model(model,
                                     test_data_loader,
                                     device,
                                     len(df_test))

    print('Test Accuracy :', test_acc)
    print('Test Loss :', test_loss)
    class_names = ['no hate', 'hate']
    # call prediction function to get actual predictions of test set 
    y_review_texts, y_pred, y_pred_probs, y_test = get_predictions(model, test_data_loader)
    
    print(classification_report(y_test, y_pred, target_names=class_names))
    report = classification_report(y_test, y_pred, target_names=class_names,output_dict=True)
    report['accuracy'] = {'precision': None,
        'recall': None,
        'f1-score': report['accuracy'],
        'support': None}
    report = pd.DataFrame(report).transpose()
    report_lst.append(report)
    cm = confusion_matrix(y_test, y_pred)
    df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
    #df_cm.to_csv('cm_hate.csv')

    show_confusion_matrix(df_cm)
    